In [1]:
from contextlib import asynccontextmanager
from fastapi import FastAPI, HTTPException
from fastapi.responses import JSONResponse, FileResponse
from pydantic import BaseModel
from seleniumbase import Driver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os
import time
import re
import subprocess
import threading
import logging
import requests
import atexit
import signal
from typing import Optional, Dict, Any
from captcha_solver import dynamic_captcha

In [ ]:
	
Token_APIKEY= "02aqx0uj628phyfz0qujvfft5w7dcyw0ra4hfdh9"
import requests
 
def get_proxies():
    response = requests.get(
        "https://proxy.webshare.io/api/v2/proxy/list/?mode=direct&page=1&page_size=25",
        headers={"Authorization": Token_APIKEY}
    )

    data = response.json()
    print(data)
    return data.get("results", [])

In [2]:

def capsolver(site_key):
    payload = {
        "clientKey": 'CAP-B5AF60896E5D3460063F4148663FAD01',
        "task": {
            "type": "AntiCloudflareTask",
            "websiteURL": "https://autodoc.co.uk/",
            "proxy": "64.137.96.74:6641:mdazxles:8kowrxvetpe2"
        }
    }
    res = requests.post("https://api.capsolver.com/createTask", json=payload)
    resp = res.json()
    task_id = resp.get("taskId")
    if not task_id:
        print("Failed to create task:", res.text)
        return
    print(f"Got taskId: {task_id} / Getting result...")
 
    while True:
        time.sleep(1)  # delay
        payload = {"clientKey": 'CAP-B5AF60896E5D3460063F4148663FAD01', "taskId": task_id}
        res = requests.post("https://api.capsolver.com/getTaskResult", json=payload)
        resp = res.json()
        status = resp.get("status")
        if status == "ready":
            return resp.get("solution", {})
        if status == "failed" or resp.get("errorId"):
            print("Solve failed! response:", res.text)
            return

PROXY = "mdazxles:8kowrxvetpe2@64.137.96.74:6641"

def start(proxy):
    """Spin up Chrome with authenticated proxy (SeleniumBase)."""
    
    driver = Driver(
        uc=True,
        headless=False,
        proxy=proxy
    )

    wait = WebDriverWait(driver, 15)

    try:
        service_pid = driver.service.process.pid
    except Exception:
        service_pid = None

    return driver
    # logger.info(f"✅ [inst-{id}] Chrome ready (service PID: {_service_pid})")
def solve_captcha(driver):
    driver.get(f"https://autodoc.co.uk/")




In [ ]:
def get_working_driver():
    proxies = get_proxies()

    for proxy_data in proxies:
        if not proxy_data.get("valid"):
            continue

        ip = proxy_data["proxy_address"]
        port = proxy_data["port"]
        username = proxy_data["username"]
        password = proxy_data["password"]

        proxy_string = f"{username}:{password}@{ip}:{port}"

        print(f"Trying proxy: {ip}:{port}")

        try:
            driver = start(proxy_string)

            # Test proxy by checking external IP
            driver.get("https://autodoc.co.uk/")

            body = driver.page_source.strip()
            print("Current IP:", body)

            if ip in body:
                print("✅ Proxy works!")
                return driver
            else:
                print("❌ Proxy IP mismatch")
                driver.quit()

        except Exception as e:
            print("❌ Failed:", e)
            try:
                driver.quit()
            except:
                pass

    raise Exception("No working proxies found.")

In [ ]:
driver = get_working_driver()

In [3]:
PROXY = "mdazxles:8kowrxvetpe2@64.137.96.74:6641"
driver =start(PROXY)
# a.solve_captcha('0x4AAAAAAAAjq6WYeRDKmebM')


In [4]:
solve_captcha(driver)

In [ ]:
capsolver("0x4AAAAAAAAjq6WYeRDKmebM")